In [1]:
%pylab inline
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

Populating the interactive namespace from numpy and matplotlib


In [2]:
import warnings
warnings.filterwarnings('ignore')

Download master index files

In [ ]:
for yr in range(2013,2018):
    for qtr in range(1,5):
        url = 'https://www.sec.gov/Archives/edgar/full-index/'+str(yr)+'/QTR'+str(qtr)+'/master.idx'
        #print(url)
        f = requests.get(url)
        fn = 'MasterIndex' + str(yr) + '_' + str(qtr) + '.idx'
        open(fn,'wb').write(f.content)

f.close()

Extracting 13D forms

In [ ]:
for yr in range(2017,2018):
    for qr in range(1,5):
        df = pd.read_table('MasterIndex'+str(yr)+'_'+str(qr)+'.idx', encoding='utf-8')
        file = df.iloc[6:].reset_index(drop=True)
        file = file.rename(columns={'Description:           Master Index of EDGAR Dissemination Feed':'tbr'})
        file = file['tbr'].str.split('|',expand=True)
        file = file.rename(columns={0:'cik',1:'Company_Name',2:'Form_Type',3:'Date_Filed',4:'Filename'})
        f13d = file[file.Form_Type=='SC 13D']
        f13d['year']=f13d.Date_Filed.apply(lambda x: int(x[:4]))
        f13d.cik = f13d.cik.astype('int')
    print(f13d)

2017 Q1

In [3]:
file = pd.read_table('MasterIndex2017_1.idx')
file = file.iloc[6:].reset_index(drop=True)
file = file.rename(columns={'Description:           Master Index of EDGAR Dissemination Feed':'tbr'})
file = file['tbr'].str.split('|',expand=True)
file = file.rename(columns={0:'cik',1:'Company_Name',2:'Form_Type',3:'Date_Filed',4:'Filename'})
f13d = file[file.Form_Type=='SC 13D']
f13d['year']=f13d.Date_Filed.apply(lambda x: int(x[:4]))
f13d.cik = f13d.cik.astype('int')

In [4]:
f13d.head()

,cik,Company_Name,Form_Type,Date_Filed,Filename,year
23,1000045,NICHOLAS FINANCIAL INC,SC 13D,2017-03-17,edgar/data/1000045/0001398344-17-003653.txt,2017
891,1001085,BROOKFIELD ASSET MANAGEMENT INC.,SC 13D,2017-02-17,edgar/data/1001085/0001104659-17-010211.txt,2017
1142,1001907,ASTROTECH Corp,SC 13D,2017-01-10,edgar/data/1001907/0001001907-17-000002.txt,2017
1851,1004724,"ADCARE HEALTH SYSTEMS, INC",SC 13D,2017-01-19,edgar/data/1004724/0001387749-17-000004.txt,2017
2306,1005699,MAGICJACK VOCALTEC LTD,SC 13D,2017-03-20,edgar/data/1005699/0001580695-17-000176.txt,2017


In [5]:
f13d.shape

(739, 6)

In [6]:
f13d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 739 entries, 23 to 311572
Data columns (total 6 columns):
cik             739 non-null int64
Company_Name    739 non-null object
Form_Type       739 non-null object
Date_Filed      739 non-null object
Filename        739 non-null object
year            739 non-null int64
dtypes: int64(2), object(4)
memory usage: 40.4+ KB


Extract Target CIK

In [7]:
def extracttargetcik(n):
    url = 'https://www.sec.gov/Archives/'+ str(n)
    f = requests.get(url)
    t = BeautifulSoup(f.content,'lxml').get_text()
    meep = re.search('SUBJECT COMPANY:.*', t, re.S)
    s = meep.group(0).splitlines
    d = meep.group(0).replace('\t','').splitlines()
    matching = [s for s in d if 'CENTRAL INDEX KEY:'in s]
    res = matching[0].strip('CENTRAL INDEX KEY:')
    cik = int(res)
    return cik

In [8]:
f13d['target_CIK']= f13d.Filename.apply(extracttargetcik)

In [15]:
f13d['target']=f13d.cik==f13d.target_CIK

In [17]:
f13d_t= f13d[f13d.cik==f13d.target_CIK]

In [18]:
f13d_t.head()

,cik,Company_Name,Form_Type,Date_Filed,Filename,year,target_CIK,target
23,1000045,NICHOLAS FINANCIAL INC,SC 13D,2017-03-17,edgar/data/1000045/0001398344-17-003653.txt,2017,1000045,True
891,1001085,BROOKFIELD ASSET MANAGEMENT INC.,SC 13D,2017-02-17,edgar/data/1001085/0001104659-17-010211.txt,2017,1001085,True
1142,1001907,ASTROTECH Corp,SC 13D,2017-01-10,edgar/data/1001907/0001001907-17-000002.txt,2017,1001907,True
1851,1004724,"ADCARE HEALTH SYSTEMS, INC",SC 13D,2017-01-19,edgar/data/1004724/0001387749-17-000004.txt,2017,1004724,True
2306,1005699,MAGICJACK VOCALTEC LTD,SC 13D,2017-03-20,edgar/data/1005699/0001580695-17-000176.txt,2017,1005699,True


In [19]:
f13d_t.shape

(372, 8)

In [20]:
f13d_t['cum_acq'] = f13d_t.groupby(['cik','year'])['target'].cumsum()

In [21]:
f13d_t[f13d_t.cum_acq==1]

,cik,Company_Name,Form_Type,Date_Filed,Filename,year,target_CIK,target,cum_acq
23,1000045,NICHOLAS FINANCIAL INC,SC 13D,2017-03-17,edgar/data/1000045/0001398344-17-003653.txt,2017,1000045,True,1.0
891,1001085,BROOKFIELD ASSET MANAGEMENT INC.,SC 13D,2017-02-17,edgar/data/1001085/0001104659-17-010211.txt,2017,1001085,True,1.0
1142,1001907,ASTROTECH Corp,SC 13D,2017-01-10,edgar/data/1001907/0001001907-17-000002.txt,2017,1001907,True,1.0
1851,1004724,"ADCARE HEALTH SYSTEMS, INC",SC 13D,2017-01-19,edgar/data/1004724/0001387749-17-000004.txt,2017,1004724,True,1.0
2306,1005699,MAGICJACK VOCALTEC LTD,SC 13D,2017-03-20,edgar/data/1005699/0001580695-17-000176.txt,2017,1005699,True,1.0
2845,1006424,"BEAR STATE FINANCIAL, INC.",SC 13D,2017-03-06,edgar/data/1006424/0001437749-17-003858.txt,2017,1006424,True,1.0
3581,1009829,JAKKS PACIFIC INC,SC 13D,2017-02-10,edgar/data/1009829/0001144204-17-007282.txt,2017,1009829,True,1.0
3655,1010134,ICTS INTERNATIONAL N V,SC 13D,2017-03-31,edgar/data/1010134/0001178913-17-000997.txt,2017,1010134,True,1.0
3848,1011432,AUXILIO INC,SC 13D,2017-01-20,edgar/data/1011432/0001445866-17-000056.txt,2017,1011432,True,1.0
4711,1013871,"NRG ENERGY, INC.",SC 13D,2017-01-17,edgar/data/1013871/0001013594-17-000060.txt,2017,1013871,True,1.0
